In [43]:
import gradio as gr
import requests
from transformers import pipeline
import os
from dotenv import load_dotenv

# HF pipeline for instruction-following
load_dotenv()

# Optionally, set your OpenWeatherMap key or simulate
WEATHER_API_KEY = os.environ["OPENWEATHERMAP_API_KEY"]  # Or simulate
print("Weather API Key:", WEATHER_API_KEY)  


Weather API Key: ad3a8beebfa81a2aff36f793b8789676


In [44]:
# from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# # Method 1: Using pipeline directly (simpler)
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")  
# # Example usage with pipeline
# inputs = tokenizer("You are an assistant respond politely: How are you?", return_tensors="pt")
# outputs = model.generate(**inputs)
# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(decoded_output)

In [45]:
pipe = pipeline("text2text-generation", model="google/flan-t5-base")

# Example usage with pipeline
input_text = "You are an assistant, respond politely and help me with the weather. what can you do ?"
result = pipe(input_text)
print(result[0]['generated_text'])


Device set to use mps:0


I can help you with the weather.


In [46]:


# Cell 2: Weather function
def get_weather(city):
    try:
        # Add error handling for the API request
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
        r = requests.get(url)
        r.raise_for_status()  # Raise an exception for bad status codes
        
        data = r.json()
        if 'weather' not in data or 'main' not in data:
            return f"Error: Invalid response format for {city}"
            
        desc = data['weather'][0]['description']
        temp = data['main']['temp']
    
        # return f"The weather in {city} is {desc} with a temperature of {temp}°C."
        return city,desc,temp
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing weather data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"



In [47]:
get_weather("London")

('London', 'broken clouds', 12.64)

In [48]:
def get_suggestion(weather_desc, temp):
    """Helper function to generate weather-appropriate suggestions"""
    suggestions = []
    
    # Temperature based suggestions
    if temp >= 20:
        suggestions.append("It's quite warm - don't forget sunscreen and stay hydrated!")
    elif temp <= 10:
        suggestions.append("It's cold - remember to dress warmly!")
        
    # Weather condition based suggestions
    if any(word in weather_desc.lower() for word in ['rain', 'shower', 'drizzle', 'thunderstorm']):
        suggestions.append("Take an umbrella with you!")
    elif any(word in weather_desc.lower() for word in ['clear', 'sunny', 'sun']):
        suggestions.append("Don't forget your sunglasses!")
    elif 'cloud' in weather_desc.lower():
        suggestions.append("It might be changeable weather - be prepared!")
        
    return " ".join(suggestions) if suggestions else ""

In [49]:
# # Agent logic — simulate reasoning and reflection
# def agent_response(user_input):
#     # Simplified and more direct prompt
#     prompt = f"""Question: {user_input}
#     If this is a weather question, respond with 'WEATHER' followed by the city name.
#     If not, respond naturally to the question."""
    
#     result = qa_pipeline(prompt)[0]["generated_text"].strip()
    
#     if "weather" in user_input.lower():
#         try:
#             # Improved city extraction
#             city = user_input.lower().split("weather in")[1].strip().rstrip('?').capitalize()
#             city,desc,temp = get_weather(city)

#             return f"The weather in {city} is {desc} with a temperature of {temp}°C \n {get_suggestion(desc, temp)}"
#         except Exception as e:
#             return f"I couldn't get the weather information for that location. Error: {str(e)}"
#     else:
#         return result

In [50]:
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")


Device set to use mps:0


In [ ]:


def agent_response(user_input):
    # Direct weather detection via simple pattern
    if "weather in" in user_input.lower():
        try:
            # Basic city name extraction
            #1. user_input.lower() - Converts the input text to lowercase
            #2. split("weather in")[1] - Splits the text at "weather in" and takes everything after it
            #3. strip() - Removes leading and trailing whitespace
            #4. rstrip('?') - Removes question mark from the end if present
            #5. capitalize() - Capitalizes the first letter of the city name
            city = user_input.lower().split("weather in")[1].strip().rstrip('?').capitalize()
            city, desc, temp = get_weather(city)

            return f"The weather in {city} is {desc} with a temperature of {temp}°C.\n\n{get_suggestion(desc, temp)}"
        except Exception as e:
            return f"I couldn't get the weather information for that location. Error: {str(e)}"
    
    # Fall back to model if user input isn't explicit
    
    prompt = (
        f"You are an assistant, respond politely and help me with the weather{user_input}"
    )

    result = qa_pipeline(prompt)
    # result = qa_pipeline(prompt)[0]["generated_text"]
    print(result[0]['generated_text'])
    if "WEATHER" in result[0]['generated_text'].upper():
        try:
            # city = result[0]['generated_text'].split(":")[1].strip().capitalize()
            # city, desc, temp = get_weather(city)
            city = user_input.lower().split("weather")[1].strip().rstrip('?').capitalize()
            city, desc, temp = get_weather(city)

            return f"{result[0]['generated_text']}.\n\n The weather in {city} is {desc} with a temperature of {temp}°C.\n\n{get_suggestion(desc, temp)}"
        except Exception as e:
            return f"I couldn't get the weather information for that location. Error: {str(e)}"
    else:
        return result[0]['generated_text']    
    
    

In [77]:
agent_response("what weather Athens")  # Example call to test the agent response

weather forecast


"weather forecast.\n\n The weather in Athens is clear sky with a temperature of 25.21°C.\n\nIt's quite warm - don't forget sunscreen and stay hydrated! Don't forget your sunglasses!"